# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [3]:
my_columns = ['Ticker', 'Stock Price', '1 Year Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [4]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        '1 Year Return': [yr1Return],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  1 Year Return Number of Shares to Buy
0        A      126.065      -0.064067                     N/A
1      AAL       16.025       0.084125                     N/A
2      AAP       71.730      -0.627598                     N/A
3     AAPL      185.255       0.156154                     N/A
4     ABBV      149.445       0.079759                     N/A
..     ...          ...            ...                     ...
498    YUM      135.800       0.145246                     N/A
499    ZBH      127.335       0.110630                     N/A
500   ZBRA      250.675      -0.247664                     N/A
501   ZION       37.930      -0.307222                     N/A
502    ZTS      180.885       0.035909                     N/A

[503 rows x 4 columns]


In [5]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,126.065,-0.064067,N/A
1,AAL,16.025,0.084125,N/A
2,AAP,71.730,-0.627598,N/A
3,AAPL,185.255,0.156154,N/A
4,ABBV,149.445,0.079759,N/A
...,...,...,...,...
498,YUM,135.800,0.145246,N/A
499,ZBH,127.335,0.110630,N/A
500,ZBRA,250.675,-0.247664,N/A
501,ZION,37.930,-0.307222,N/A


In [6]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
80,CAT,280.385,NaN,N/A


Filling the `NULL` cells of the `1 Year Return` column with mean value

In [7]:
final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)

In [8]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


In [9]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,126.065,-0.064067,N/A
1,AAL,16.025,0.084125,N/A
2,AAP,71.730,-0.627598,N/A
3,AAPL,185.255,0.156154,N/A
4,ABBV,149.445,0.079759,N/A
...,...,...,...,...
498,YUM,135.800,0.145246,N/A
499,ZBH,127.335,0.110630,N/A
500,ZBRA,250.675,-0.247664,N/A
501,ZION,37.930,-0.307222,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values('1 Year Return', ascending=False, inplace=True)

In [11]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [12]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,RCL,104.8000,1.639778,N/A
1,NVDA,449.1950,1.344252,N/A
2,GE,112.4915,0.935626,N/A
3,FSLR,199.0500,0.925810,N/A
4,PHM,83.8700,0.921897,N/A
5,NFLX,431.1700,0.900520,N/A
6,META,313.4100,0.874155,N/A
7,CCL,17.4750,0.788187,N/A
8,FICO,840.8050,0.772504,N/A
9,ACGL,77.1600,0.766492,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [13]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [15]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,RCL,104.8000,1.639778,1908
1,NVDA,449.1950,1.344252,445
2,GE,112.4915,0.935626,1777
3,FSLR,199.0500,0.925810,1004
4,PHM,83.8700,0.921897,2384
5,NFLX,431.1700,0.900520,463
6,META,313.4100,0.874155,638
7,CCL,17.4750,0.788187,11444
8,FICO,840.8050,0.772504,237
9,ACGL,77.1600,0.766492,2592


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is that low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Here `hqm` stands for `high-quality momentum`.

In [16]:
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    '1 Year Return',
    '1 Year Return Percentile',
    '6 Months Return',
    '6 Months Return Percentile',
    '3 Months Return',
    '3 Months Return Percentile',
    '1 Month Return',
    '1 Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [17]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='6mo')
        mnth6Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth6Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='3mo')
        mnth3Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth3Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='1mo')
        mnth1Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth1Return = np.NaN
    
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        'Number of Shares to Buy': ['N/A'],
        '1 Year Return': [yr1Return],
        '1 Year Return Percentile': ['N/A'],
        '6 Months Return': [mnth6Return],
        '6 Months Return Percentile': ['N/A'],
        '3 Months Return': [mnth3Return],
        '3 Months Return Percentile': ['N/A'],
        '1 Month Return': [mnth1Return],
        '1 Month Return Percentile': ['N/A'],
        'HQM Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    hqm_dataframe = pd.concat([hqm_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(hqm_dataframe['Ticker'])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
498     YUM
499     ZBH
500    ZBRA
501    ZION
502     ZTS
Name: Ticker, Length: 503, dtype: object


In [18]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.0500,N/A,-0.064067,N/A,-16.909856,N/A,-5.668729,N/A,5.569037,N/A,N/A
1,AAL,15.9791,N/A,0.084125,N/A,-5.557920,N/A,18.279790,N/A,-12.775108,N/A,N/A
2,AAP,71.7700,N/A,-0.627598,N/A,-51.917973,N/A,-40.099385,N/A,1.214864,N/A,N/A
3,AAPL,185.1089,N/A,0.156154,N/A,22.347740,N/A,11.801215,N/A,-3.256575,N/A,N/A
4,ABBV,149.2150,N/A,0.079759,N/A,4.990424,N/A,2.381386,N/A,9.649431,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.7500,N/A,0.145246,N/A,6.160866,N/A,0.046534,N/A,-0.665889,N/A,N/A
499,ZBH,127.0000,N/A,0.110630,N/A,-0.829496,N/A,-6.893679,N/A,-10.789548,N/A,N/A
500,ZBRA,251.9825,N/A,-0.247664,N/A,-22.395291,N/A,-4.912265,N/A,-13.955099,N/A,N/A
501,ZION,37.9550,N/A,-0.307222,N/A,-28.051563,N/A,93.798253,N/A,35.756444,N/A,N/A


In [19]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
80,CAT,280.3777,N/A,NaN,N/A,12.678249,N/A,33.932013,N/A,15.08646,N/A,N/A


In [20]:
hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)

In [21]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [22]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.0500,N/A,-0.064067,N/A,-16.909856,N/A,-5.668729,N/A,5.569037,N/A,N/A
1,AAL,15.9791,N/A,0.084125,N/A,-5.557920,N/A,18.279790,N/A,-12.775108,N/A,N/A
2,AAP,71.7700,N/A,-0.627598,N/A,-51.917973,N/A,-40.099385,N/A,1.214864,N/A,N/A
3,AAPL,185.1089,N/A,0.156154,N/A,22.347740,N/A,11.801215,N/A,-3.256575,N/A,N/A
4,ABBV,149.2150,N/A,0.079759,N/A,4.990424,N/A,2.381386,N/A,9.649431,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.7500,N/A,0.145246,N/A,6.160866,N/A,0.046534,N/A,-0.665889,N/A,N/A
499,ZBH,127.0000,N/A,0.110630,N/A,-0.829496,N/A,-6.893679,N/A,-10.789548,N/A,N/A
500,ZBRA,251.9825,N/A,-0.247664,N/A,-22.395291,N/A,-4.912265,N/A,-13.955099,N/A,N/A
501,ZION,37.9550,N/A,-0.307222,N/A,-28.051563,N/A,93.798253,N/A,35.756444,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [23]:
time_periods = [
    '1 Year',
    '6 Months',
    '3 Months',
    '1 Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[change_col],
                                                                           hqm_dataframe.loc[row, change_col]) / 100

In [24]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.0500,N/A,-0.064067,0.306163,-16.909856,0.089463,-5.668729,0.135189,5.569037,0.689861,N/A
1,AAL,15.9791,N/A,0.084125,0.590457,-5.557920,0.300199,18.279790,0.749503,-12.775108,0.027833,N/A
2,AAP,71.7700,N/A,-0.627598,0.001988,-51.917973,0.001988,-40.099385,0.001988,1.214864,0.457256,N/A
3,AAPL,185.1089,N/A,0.156154,0.719682,22.347740,0.89662,11.801215,0.582505,-3.256575,0.2167,N/A
4,ABBV,149.2150,N/A,0.079759,0.582505,4.990424,0.576541,2.381386,0.353877,9.649431,0.84493,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.7500,N/A,0.145246,0.695825,6.160866,0.614314,0.046534,0.292247,-0.665889,0.351889,N/A
499,ZBH,127.0000,N/A,0.110630,0.646123,-0.829496,0.437376,-6.893679,0.105368,-10.789548,0.047714,N/A
500,ZBRA,251.9825,N/A,-0.247664,0.073559,-22.395291,0.059642,-4.912265,0.147117,-13.955099,0.019881,N/A
501,ZION,37.9550,N/A,-0.307222,0.053678,-28.051563,0.031809,93.798253,1.0,35.756444,1.0,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [25]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [26]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.0500,N/A,-0.064067,0.306163,-16.909856,0.089463,-5.668729,0.135189,5.569037,0.689861,0.305169
1,AAL,15.9791,N/A,0.084125,0.590457,-5.557920,0.300199,18.279790,0.749503,-12.775108,0.027833,0.416998
2,AAP,71.7700,N/A,-0.627598,0.001988,-51.917973,0.001988,-40.099385,0.001988,1.214864,0.457256,0.115805
3,AAPL,185.1089,N/A,0.156154,0.719682,22.347740,0.89662,11.801215,0.582505,-3.256575,0.2167,0.603877
4,ABBV,149.2150,N/A,0.079759,0.582505,4.990424,0.576541,2.381386,0.353877,9.649431,0.84493,0.589463
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.7500,N/A,0.145246,0.695825,6.160866,0.614314,0.046534,0.292247,-0.665889,0.351889,0.488569
499,ZBH,127.0000,N/A,0.110630,0.646123,-0.829496,0.437376,-6.893679,0.105368,-10.789548,0.047714,0.309145
500,ZBRA,251.9825,N/A,-0.247664,0.073559,-22.395291,0.059642,-4.912265,0.147117,-13.955099,0.019881,0.07505
501,ZION,37.9550,N/A,-0.307222,0.053678,-28.051563,0.031809,93.798253,1.0,35.756444,1.0,0.521372


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [27]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [28]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [29]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,CARR,57.6300,N/A,0.387297,0.918489,25.923295,0.926441,40.759022,0.978131,17.069453,0.962227,0.946322
1,MPC,139.3900,N/A,0.509865,0.962227,21.264874,0.890656,33.582173,0.946322,19.946649,0.982107,0.945328
2,ANET,180.2500,N/A,0.403221,0.926441,38.985270,0.980119,33.183098,0.942346,12.882019,0.914513,0.940855
3,NVDA,451.6700,N/A,1.344252,0.998012,114.231818,1.0,63.891149,0.994036,6.734882,0.743539,0.933897
4,ON,102.7250,N/A,0.518557,0.966203,27.767409,0.944334,31.986376,0.936382,10.516409,0.870775,0.929423
5,PHM,84.0500,N/A,0.921897,0.992048,46.961988,0.99006,26.913105,0.894632,8.985996,0.817097,0.923459
6,LRCX,695.1000,N/A,0.333462,0.888668,34.282694,0.968191,36.072153,0.964215,10.582576,0.872763,0.923459
7,META,314.3429,N/A,0.874155,0.988072,68.947060,0.998012,34.610693,0.956262,6.786022,0.747515,0.922465
8,ETN,218.3100,N/A,0.460478,0.94831,35.825718,0.970179,30.272106,0.912525,9.758675,0.846918,0.919483
9,CEG,102.9550,N/A,0.361238,0.904573,24.538563,0.918489,31.469290,0.926441,11.737576,0.898608,0.912028


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [30]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [31]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

200000.0

In [32]:
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Stock Price'])

hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,CARR,57.6300,3470,0.387297,0.918489,25.923295,0.926441,40.759022,0.978131,17.069453,0.962227,0.946322
1,MPC,139.3900,1434,0.509865,0.962227,21.264874,0.890656,33.582173,0.946322,19.946649,0.982107,0.945328
2,ANET,180.2500,1109,0.403221,0.926441,38.985270,0.980119,33.183098,0.942346,12.882019,0.914513,0.940855
3,NVDA,451.6700,442,1.344252,0.998012,114.231818,1.0,63.891149,0.994036,6.734882,0.743539,0.933897
4,ON,102.7250,1946,0.518557,0.966203,27.767409,0.944334,31.986376,0.936382,10.516409,0.870775,0.929423
5,PHM,84.0500,2379,0.921897,0.992048,46.961988,0.99006,26.913105,0.894632,8.985996,0.817097,0.923459
6,LRCX,695.1000,287,0.333462,0.888668,34.282694,0.968191,36.072153,0.964215,10.582576,0.872763,0.923459
7,META,314.3429,636,0.874155,0.988072,68.947060,0.998012,34.610693,0.956262,6.786022,0.747515,0.922465
8,ETN,218.3100,916,0.460478,0.94831,35.825718,0.970179,30.272106,0.912525,9.758675,0.846918,0.919483
9,CEG,102.9550,1942,0.361238,0.904573,24.538563,0.918489,31.469290,0.926441,11.737576,0.898608,0.912028


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [33]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Formatting Our Excel Output

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [36]:
writer.close()